In [1]:
import pandas as pd
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import json
from fbprophet import Prophet

with open('krx_total_list.txt','r') as f:
    tickers = json.loads(f.read())
len(tickers)

2085

In [2]:
tickerlist = list(tickers.keys())
len(tickerlist)

2085

# Multiprocess Iteration

In [3]:
from multiprocessing import Pool
from time import time as t
start = t()

n1 = 160
n2 = 164 

num_list = list(range(n1,n2))
log=[]
print('make prediction : {} ~ {}'.format(n1,n2))
def sub_func(i):
    global log
    cnt=i
    v = tickers[tickerlist[i]]
    ticker = tickerlist[i]+'.'+v['마켓']
    try:
        data = pd.read_pickle('./data5y/{}-test'.format(ticker))
        m = Prophet(daily_seasonality=True)
        df = pd.DataFrame({'ds':data.index, 'y':data['Close']})
        m.fit(df);
        future = m.make_future_dataframe(periods=365)
        forecast = m.predict(future)
        m.plot(forecast).savefig('./graph5y_predict/{}-test.png'.format(ticker))
        log.append((cnt,ticker,'plotted.',v['회사명']))
        print(cnt,ticker,'.',end='')
    except Exception as e:
        log.append((cnt,ticker,'failed.',e,v['회사명']))
        print(cnt,ticker,'failed',e,v['회사명'])
        pass

pool = Pool(processes=2)
pool.map(sub_func, num_list)
print('prediction finished.')
filename = 'log-predict-{}-{}.txt'.format(n1,n2)
with open('./graph5y_predict/'+filename, 'w') as f:
    for i in log:
        f.write("{}\n".format(i))
    print(filename,'file saved!')
    
print('time:', t()-start)

make prediction : 160 ~ 164


/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.p

prediction finished.
log-predict-160-164.txt file saved!
time: 23.315347909927368
